# JB금융그룹 2차 데이터 분석 경진대회 Track1
- 과제1 : 개인 대출 부도 분석 미 예측 모델링

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import calendar
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import roc_auc_score
from impyute.imputation.cs import mice, fast_knn
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import datawig

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import  sklearn.svm as svm
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import xgboost as xgb 
from imblearn.over_sampling import SMOTE

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pd.options.display.max_columns =None

In [ ]:
pwd

## 1.0.0 READ DATA

In [ ]:
df = pd.read_csv('DATA/loan_customer.csv')

In [ ]:
n_col=[]
for i in df.columns:
    n_col.append(i.upper())

In [ ]:
df.columns = n_col

## 2.0.0 EDA

In [ ]:
df.info()

- sk_id_curr : 대출 식별번호 
- name_contract_typ  : Cash loans: 일반 대출, Revolving loans: 마이너스 대출
- code_gender	        성별
- flag_own_realty	    자가부동산 보유 여부
- cnt_children	        자녀 수
- amt_income_total	   소득 금액(USD)
- amt_credit	  신용대출 금액(USD)
- name_income_type	소득 유형
- name_education_type	학력
- name_family_status	결혼 상태
- name_housing_type	주거 형태
- region_population_relative	거주 지역의 표준화된 인구
- days_birth	대출 신청 시점에서의 일단위 나이(기준 시점 대비 상대적인 시간)
- days_employed	대출 신청 시점에서의 고객의 근무일수(기준 시점 대비 상대적인 시간)
- days_registration	대출 신청 이전에 고객이 등록정보를 변경한 시점 사이의 일수(기준 시점 대비 상대적인 시간)
- days_id_publish	대출 신청 이전에 고객이 신분증을 변경한 시점 사이의 일수(기준 시점 대비 상대적인 시간)
- flag_mobil	휴대폰 번호 등록 여부
- flag_emp_phone	고용주 연락처 등록 여부
- flag_work_phone	직장 전화번호 등록 여부
- flag_phone	집 전화번호 등록 여부
- flag_email	이메일 주소 등록 여부
- region_rating_client	거주지 등급
- weekday_appr_process_start	대출 신청 요일
- hour_appr_process_start	대출 신청 시간대
- reg_city_not_live_city	고객의 등록지 주소와 실거주지 일치 여부
- reg_city_not_work_city	고객의 등록지 주소와 직장 주소 일치 여부
- live_city_not_work_city	고객의 실거주지와 직장 주소 일치 여부
- organization_type	근무 회사 유형
- flag_document_2	문서 2 제공 여부
- flag_document_3	문서 3 제공 여부
- flag_document_4	문서 4 제공 여부
- cb_score_1	신용점수 1
- cb_score_2	신용점수 2
- cb_score_3	신용점수 3
- target	타겟 변수 (1 - 대출 신청자 중 지정한 일수 동안 한번 이상 대출 분할 지불이 지연된 경우, 0 - 그 외의 경우)



### 2.1.0 column 타입변경 

In [ ]:
df_st1 = df.copy()

In [ ]:
df_st1 = df_st1.drop('SK_ID_CURR',axis=1)


- 라벨인코딩된 형태로 존재하는 변수 명목형으로 변경

In [ ]:
df_st1['FLAG_MOBIL'] = df_st1['FLAG_MOBIL'].astype('str')
df_st1['FLAG_EMP_PHONE'] = df_st1['FLAG_EMP_PHONE'].astype('str')
df_st1['FLAG_WORK_PHONE'] = df_st1['FLAG_WORK_PHONE'].astype('str')
df_st1['FLAG_PHONE'] = df_st1['FLAG_PHONE'].astype('str')
df_st1['FLAG_EMAIL'] = df_st1['FLAG_EMAIL'].astype('str')
df_st1['REGION_RATING_CLIENT'] = df_st1['REGION_RATING_CLIENT'].astype('str')
df_st1['HOUR_APPR_PROCESS_START'] = df_st1['HOUR_APPR_PROCESS_START'].astype('str')
df_st1['REG_CITY_NOT_LIVE_CITY'] = df_st1['REG_CITY_NOT_LIVE_CITY'].astype('str')
df_st1['REG_CITY_NOT_WORK_CITY'] = df_st1['REG_CITY_NOT_WORK_CITY'].astype('str')
df_st1['LIVE_CITY_NOT_WORK_CITY'] = df_st1['LIVE_CITY_NOT_WORK_CITY'].astype('str')
df_st1['FLAG_DOCUMENT_2'] = df_st1['FLAG_DOCUMENT_2'].astype('str')
df_st1['FLAG_DOCUMENT_3'] = df_st1['FLAG_DOCUMENT_3'].astype('str')
df_st1['FLAG_DOCUMENT_4'] = df_st1['FLAG_DOCUMENT_4'].astype('str')


In [ ]:
df_st1['CNT_CHILDREN']               = df_st1['CNT_CHILDREN'].astype('int')
df_st1['AMT_INCOME_TOTAL']           = df_st1['AMT_INCOME_TOTAL'].astype('float64')
df_st1['AMT_CREDIT']                 = df_st1['AMT_CREDIT'].astype('float64')
df_st1['REGION_POPULATION_RELATIVE'] = df_st1['REGION_POPULATION_RELATIVE'].astype('float64')
df_st1['DAYS_BIRTH']                 = df_st1['DAYS_BIRTH'].astype('float64')
df_st1['DAYS_EMPLOYED']              = df_st1['DAYS_EMPLOYED'].astype('float64')
df_st1['DAYS_REGISTRATION']          = df_st1['DAYS_REGISTRATION'].astype('float64')
df_st1['DAYS_ID_PUBLISH']            = df_st1['DAYS_ID_PUBLISH'].astype('float64')
df_st1['DAYS_EMPLOYED']              = df_st1['DAYS_EMPLOYED'].astype('float64')
df_st1['CB_SCORE_1']                 = df_st1['CB_SCORE_1'].astype('float64')
df_st1['CB_SCORE_2']                 = df_st1['CB_SCORE_2'].astype('float64')
df_st1['CB_SCORE_3']                 = df_st1['CB_SCORE_3'].astype('float64')

In [ ]:
df_st1['TARGET'] = df_st1['TARGET'].astype('int')

In [ ]:
df_obj = ['NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_REALTY','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE',
        'FLAG_MOBIL','FLAG_EMP_PHONE','FLAG_WORK_PHONE','FLAG_PHONE','FLAG_EMAIL','REGION_RATING_CLIENT','WEEKDAY_APPR_PROCESS_START','HOUR_APPR_PROCESS_START',
        'REG_CITY_NOT_LIVE_CITY','REG_CITY_NOT_WORK_CITY','LIVE_CITY_NOT_WORK_CITY','ORGANIZATION_TYPE','FLAG_DOCUMENT_2','FLAG_DOCUMENT_3','FLAG_DOCUMENT_4']

In [ ]:
df_num = list(set(df.columns) - set(df_obj)-set(['TARGET','SK_ID_CURR']))

In [ ]:
df_num

In [ ]:
df_obj_non = ['CODE_GENDER','FLAG_OWN_REALTY','ORGANIZATION_TYPE']
df_num_non = ['AMT_CREDIT','REGION_POPULATION_RELATIVE','DAYS_BIRTH','DAYS_EMPLOYED','DAYS_REGISTRATION','DAYS_ID_PUBLISH','CB_SCORE_1','CB_SCORE_2','CB_SCORE_3']
df_num_notn = list(set(df_num)-set(df_num_non))
df_obj_notn = list(set(df_obj)-set(df_obj_non))
df_notn = list(set(df.columns) - set(df_num_non)-set(df_obj_non)-set(['TARGET','SK_ID_CURR']) -set(['FLAG_MOBIL','FLAG_DOCUMENT_2','FLAG_DOCUMENT_4']))

- 변수들 타입점검

In [ ]:
df_st1.loc[:,df_num].info()

In [ ]:
df_st1.loc[:,df_num_non].info()

In [ ]:
df_st1.loc[:,df_num_notn].info()

In [ ]:
df_st1.loc[:,df_obj].info()

In [ ]:
df_st1.loc[:,df_obj_non].info()

In [ ]:
df_st1.loc[:,df_obj_notn].info()

### 2.2.0 시각화

In [ ]:
plt.style.use('ggplot')

In [ ]:
plt.set_loglevel('WARNING')

- 명목형 변수들의 빈도그래프와 각 값의 부도율

In [ ]:
fig, ax = plt.subplots(8,3,figsize =(22,50))
a =0
for i in df_obj:
    a_1 = a%3
    a_2 = a//3
    
    ax[a_2,a_1].plot((df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values(),color = 'blue')
    ax[a_2,a_1].set_ylim(0,0.2)
    ax2 = ax[a_2,a_1].twinx()
    sns.countplot(x = i ,palette='Set2' ,data =df_st1 ,order = (df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values().index
                  ,ax = ax2)
    ax[a_2,a_1].set_zorder(1)
    ax2.set_zorder(-1)
    ax[a_2,a_1].patch.set_visible(False)
    ax[a_2,a_1].tick_params(axis ='x',
                    rotation = 45,
                   )
    plt.title(i)
    plt.subplots_adjust(hspace = 0.5,
                        wspace = 0.5)
    a +=1

plt.show()
         

- FLAG_OWN_REALY, WEEKDAY 변수는 사용하기 어려워보인다.
- FLAG_MOBIL, FLAG_DOCUMENT_2,FLAG_DOCUMENT_4 변수는 단일값, 지나친 불균형으로 사용하기 어려워보인다. 
- NAME_HOUSING_TYPE, NAME_FAMILY_ STATUS 변수에서도 불균형을 볼 수 있다.

In [ ]:
df_st1 = df_st1.drop(['FLAG_MOBIL','FLAG_DOCUMENT_2','FLAG_DOCUMENT_4'],axis=1)

In [ ]:
print(df_st1[df_st1['FLAG_OWN_REALTY'] =='Y']['TARGET'].sum()/df_st1[df_st1['FLAG_OWN_REALTY'] =='Y']['TARGET'].count())
print(df_st1[df_st1['FLAG_OWN_REALTY'] =='N']['TARGET'].sum()/df_st1[df_st1['FLAG_OWN_REALTY'] =='N']['TARGET'].count())

In [ ]:
fig, ax = plt.subplots(1,1,figsize =(20,5))
i= 'ORGANIZATION_TYPE'
    
ax.plot((df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values(),color = 'blue')
ax2 = ax.twinx()
sns.countplot(x = i ,palette='Set2' ,data =df_st1 ,order = (df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values().index,ax = ax2)
ax.set_zorder(1)
ax2.set_zorder(-1)
ax.patch.set_visible(False)
ax.tick_params(axis ='x',
                rotation = 90,
               )
plt.title(i)
plt.subplots_adjust(hspace = 0.5,
                    wspace = 0.5)

plt.show()

- XNA 값을 결측치로 볼 것 인가 하나의 값으로 볼 것 인가
- 유효값이 너무 많으므로 어느정도 수준으로 그룹화가 필요해 보인다.

In [ ]:
df_st1_0 = df_st1[df_st1['TARGET']==0].reset_index(drop=True)
df_st1_1 = df_st1[df_st1['TARGET']==1].reset_index(drop=True)


In [ ]:
fig, ax = plt.subplots(4,3,figsize=(15, 20))
a=0
for i in df_num :
    a_1 = a%3
    a_2 = a//3
    
    ax[a_2,a_1].hist(df_st1_0[i])
    ax[a_2,a_1].set_title(i)
    plt.subplots_adjust(hspace = 0.5,
                        wspace = 0.5)
    
    a +=1

In [ ]:
fig, ax = plt.subplots(6,4,figsize=(18, 24))
a=0
for i in df_num :
    a_1 = a%4
    a_2 = a//4
    
    ax[a_2,a_1].hist(df_st1_0[i])
    ax[a_2,a_1+1].hist(df_st1_1[i])
    ax[a_2,a_1].set_title(i+'_0')
    ax[a_2,a_1+1].set_title(i+'_1')
    plt.subplots_adjust(hspace = 0.5,
                        wspace = 0.5)
    
    a +=2

- XNA 값을 결측치로 볼 것 인가 하나의 값으로 볼 것 인가
- AMT_INCOME_TOTAL, DAYS_EMPLOYED 변수의 이상치 확인
- CB_SCORE 들은 형태가 아름답다.
- AMT_INCOME_TOTAL 의 이상치는 하나의 값이 너무나 크다

## 3.0.0 탐색 및 전처리

### 3.1.0 특이 변수 탐색

- 먼저 너무많은 결측이나 의미없는 변수는 지워주자

#### 3.1.1 AMT_INCOME_TOTAL

In [ ]:
plt.boxplot(df_st1['AMT_INCOME_TOTAL'])
plt.show()

- 지나치게 큰 값이 있다. 

In [ ]:
df_st1['AMT_INCOME_TOTAL'].sort_values(ascending=False)

- 2번째 값보다 1 큰 값으로 변경

In [ ]:
df_st1.loc[df_st1['AMT_INCOME_TOTAL']==df_st1['AMT_INCOME_TOTAL'].max(),'AMT_INCOME_TOTAL'] = 3375001.0

In [ ]:
plt.boxplot(df_st1['AMT_INCOME_TOTAL'])
plt.show()

- 여전히 이상하다

In [ ]:
plt.boxplot(df_st1.loc[df_st1['AMT_INCOME_TOTAL']<=1000000,'AMT_INCOME_TOTAL'])
plt.show()

 - 1000000 이상의 값들에 대해서 처리해 주는것이 적절해 보임

In [ ]:
df_st1.loc[df_st1['AMT_INCOME_TOTAL']>=1000000,'AMT_INCOME_TOTAL'] = 1000000

In [ ]:
plt.boxplot(df_st1['AMT_INCOME_TOTAL'])
plt.show()

In [ ]:
df_st1.loc[:,'AMT_INCOME_TOTAL'].hist()

In [ ]:
np.log(df_st1.loc[:,'AMT_INCOME_TOTAL']).hist()

- log 변환한 변수를 사용하자!

In [ ]:
df_st1_log = df_st1.copy()

In [ ]:
df_st1_log.loc[:,'AMT_INCOME_TOTAL'] = np.log(df_st1.loc[:,'AMT_INCOME_TOTAL'])

#### 3.1.2 ORGANIZATION_TYPE

In [ ]:
fig, ax = plt.subplots(1,1,figsize =(20,5))
i= 'ORGANIZATION_TYPE'
    
ax.plot((df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values(),color = 'blue')
ax2 = ax.twinx()
sns.countplot(x = i ,palette='Set2' ,data =df_st1 ,order = (df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values().index,ax = ax2)
ax.set_zorder(1)
ax2.set_zorder(-1)
ax.patch.set_visible(False)
ax.tick_params(axis ='x',
                rotation = 90,
               )
plt.title(i)
plt.subplots_adjust(hspace = 0.5,
                    wspace = 0.5)

plt.show()

- 산업별로 군집하자
- XNA 를 처리하자

In [ ]:
df_st1[df_st1['ORGANIZATION_TYPE']=='XNA']['NAME_INCOME_TYPE']

- 연금수령자의 경우 ORGANIZATION_TYPE이 전부 XNA 이므로 하나의 카테고리로 보면 될 것 같다!

In [ ]:
df_st1['ORGANIZATION_TYPE2'] = df_st1['ORGANIZATION_TYPE'].str.split().str[0]


In [ ]:
fig, ax = plt.subplots(1,1,figsize =(20,5))
i= 'ORGANIZATION_TYPE2'
    
ax.plot((df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values(),color = 'blue')
ax2 = ax.twinx()
sns.countplot(x = i ,palette='Set2' ,data =df_st1 ,order = (df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values().index,ax = ax2)
ax.set_zorder(1)
ax2.set_zorder(-1)
ax.patch.set_visible(False)
ax.tick_params(axis ='x',
                rotation = 90,
               )
plt.title(i)
plt.subplots_adjust(hspace = 0.5,
                    wspace = 0.5)

plt.show()

- group1 religion~ university + 기타
- group2 XNA
- group3 medicine ~ telecom
- group4 business
- group5 restaurant ~ trade
- group6 self_employed
- group7 security~


In [ ]:
df_st1['ORGANIZATION_TYPE3'] = ''

In [ ]:
df_st1.loc[df_st1['ORGANIZATION_TYPE2'].isin((df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values().index[:12]),'ORGANIZATION_TYPE3'] = 'group1'
df_st1.loc[df_st1['ORGANIZATION_TYPE2'].isin([(df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values().index[12]]),'ORGANIZATION_TYPE3'] = 'group2'
df_st1.loc[df_st1['ORGANIZATION_TYPE2'].isin((df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values().index[13:19]),'ORGANIZATION_TYPE3'] = 'group3'
df_st1.loc[df_st1['ORGANIZATION_TYPE2'].isin([(df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values().index[19]]),'ORGANIZATION_TYPE3'] = 'group4'
df_st1.loc[df_st1['ORGANIZATION_TYPE2'].isin((df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values().index[20:26]),'ORGANIZATION_TYPE3'] = 'group5'
df_st1.loc[df_st1['ORGANIZATION_TYPE2'].isin([(df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values().index[26]]),'ORGANIZATION_TYPE3'] = 'group6'
df_st1.loc[df_st1['ORGANIZATION_TYPE2'].isin((df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values().index[27:]),'ORGANIZATION_TYPE3'] = 'group7'

In [ ]:
df_st1['ORGANIZATION_TYPE3'].value_counts()

In [ ]:
fig, ax = plt.subplots(1,1,figsize =(20,5))
i= 'ORGANIZATION_TYPE3'
    
ax.plot((df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values(),color = 'blue')
ax2 = ax.twinx()
sns.countplot(x = i ,palette='Set2' ,data =df_st1 ,order = (df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values().index,ax = ax2)
ax.set_zorder(1)
ax2.set_zorder(-1)
ax.patch.set_visible(False)
ax.tick_params(axis ='x',
                rotation = 90,
               )
plt.title(i)
plt.subplots_adjust(hspace = 0.5,
                    wspace = 0.5)

plt.show()

In [ ]:
df_st1 = df_st1.drop(['ORGANIZATION_TYPE','ORGANIZATION_TYPE2'],axis=1)

#### 3.1.3 FALG_ 변수들

In [ ]:
df_st1.columns

In [ ]:
FLAG = ['FLAG_EMP_PHONE','FLAG_WORK_PHONE','FLAG_PHONE','FLAG_EMAIL']

In [ ]:
fig, ax = plt.subplots(2,2,figsize =(15,10))
a =0
for i in FLAG:
    a_1 = a%2
    a_2 = a//2
    
    ax[a_2,a_1].plot((df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values(),color = 'blue')
    ax[a_2,a_1].set_ylim(0,0.2)
    ax2 = ax[a_2,a_1].twinx()
    sns.countplot(x = i ,palette='Set2' ,data =df_st1 ,order = (df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values().index
                  ,ax = ax2)
    ax[a_2,a_1].set_zorder(1)
    ax2.set_zorder(-1)
    ax[a_2,a_1].patch.set_visible(False)
    ax[a_2,a_1].tick_params(axis ='x',
                    rotation = 45,
                   )
    plt.title(i)
    plt.subplots_adjust(hspace = 0.5,
                        wspace = 0.5)
    a +=1

plt.show()

- FLAG EMP, WORK 는 등록한사람이 부도 발생확률이 높았고 FLAG_PHONE, EMAIL 은 등록하지 않은사람이 높았다.
- 각각 더해줘서 두개의 변수로 만들자

In [ ]:
df_st1['FLAG_WORK_INFO'] = df_st1['FLAG_EMP_PHONE'].astype(int) + df_st1['FLAG_WORK_PHONE'].astype(int)

In [ ]:
df_st1['FLAG_WORK_INFO'] = df_st1['FLAG_WORK_INFO'].astype(str)

In [ ]:
fig, ax = plt.subplots(1,1,figsize =(10,5))
i= 'FLAG_WORK_INFO'
    
ax.plot((df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values(),color = 'blue')
ax2 = ax.twinx()
sns.countplot(x = i ,palette='Set2' ,data =df_st1 ,order = (df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values().index,ax = ax2)
ax.set_zorder(1)
ax2.set_zorder(-1)
ax.patch.set_visible(False)
ax.tick_params(axis ='x',
                rotation = 90,
               )
plt.title(i)
plt.subplots_adjust(hspace = 0.5,
                    wspace = 0.5)

plt.show()

In [ ]:
df_st1['FLAG_IND_INFO'] = df_st1['FLAG_PHONE'].astype(int) + df_st1['FLAG_EMAIL'].astype(int)

In [ ]:
df_st1['FLAG_IND_INFO'] = df_st1['FLAG_IND_INFO'].astype(str)

In [ ]:
fig, ax = plt.subplots(1,1,figsize =(10,5))
i= 'FLAG_IND_INFO'
    
ax.plot((df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values(),color = 'blue')
ax2 = ax.twinx()
sns.countplot(x = i ,palette='Set2' ,data =df_st1 ,order = (df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values().index,ax = ax2)
ax.set_zorder(1)
ax2.set_zorder(-1)
ax.patch.set_visible(False)
ax.tick_params(axis ='x',
                rotation = 90,
               )
plt.title(i)
plt.subplots_adjust(hspace = 0.5,
                    wspace = 0.5)

plt.show()

In [ ]:
df_st1 = df_st1.drop(['FLAG_EMP_PHONE','FLAG_WORK_PHONE','FLAG_PHONE','FLAG_EMAIL'],axis=1)

In [ ]:
df_st1.info()

#### 3.1.4 CODE_GENDER

In [ ]:
fig, ax = plt.subplots(1,1,figsize =(10,5))
i= 'CODE_GENDER'
    
ax.plot((df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values(),color = 'blue')
ax2 = ax.twinx()
sns.countplot(x = i ,palette='Set2' ,data =df_st1 ,order = (df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values().index,ax = ax2)
ax.set_zorder(1)
ax2.set_zorder(-1)
ax.patch.set_visible(False)
ax.tick_params(axis ='x',
                rotation = 90,
               )
plt.title(i)
plt.subplots_adjust(hspace = 0.5,
                    wspace = 0.5)

plt.show()

In [ ]:
df_st1.loc[df_st1['CODE_GENDER']=='XNA','CODE_GENDER'] = np.NaN

In [ ]:
df_F = df_st1.loc[df_st1['CODE_GENDER']=='F',:]
df_M = df_st1.loc[df_st1['CODE_GENDER']=='M',:]


In [ ]:
plt.set_loglevel('WARNING')

In [ ]:
fig, ax = plt.subplots(9,4,figsize =(22,40))
a =0
for i in df_F.columns[df_F.dtypes == 'object']:
    a_1 = a%4
    a_2 = a//4
    
    ax[a_2,a_1].plot((df_M.groupby(i)['TARGET'].sum()/df_M.groupby(i)['TARGET'].count()).sort_values(),color = 'blue')
    ax[a_2,a_1].set_ylim(0,0.2)
    ax[a_2,a_1+1].plot((df_F.groupby(i)['TARGET'].sum()/df_F.groupby(i)['TARGET'].count()).sort_values(),color = 'blue')
    ax[a_2,a_1+1].set_ylim(0,0.2)
    ax2 = ax[a_2,a_1].twinx()
    ax3 = ax[a_2,a_1+1].twinx()
    sns.countplot(x = i ,palette='Set2' ,data =df_M ,order = (df_st1.groupby(i)['TARGET'].sum()/df_M.groupby(i)['TARGET'].count()).sort_values().index
                  ,ax = ax2)
    sns.countplot(x = i ,palette='Set2' ,data =df_F ,order = (df_st1.groupby(i)['TARGET'].sum()/df_F.groupby(i)['TARGET'].count()).sort_values().index
                  ,ax = ax3)
    
    ax[a_2,a_1].set_zorder(1)
    ax[a_2,a_1+1].set_zorder(1)
    
    ax2.set_zorder(-1)
    ax3.set_zorder(-1)
    
    ax[a_2,a_1].patch.set_visible(False)
    ax[a_2,a_1+1].patch.set_visible(False)
    
    
    ax[a_2,a_1].tick_params(axis ='x',
                    rotation = 45,
                   )
    ax[a_2,a_1+1].tick_params(axis ='x',
                    rotation = 45,
                   )
    

    plt.title(i)
    plt.subplots_adjust(hspace = 0.5,
                        wspace = 0.5)
    a +=2

plt.show()

- 특정 패턴을 찾기 어려워 datawig의 딥러닝을 활용한 성별예측

In [ ]:
df_gender =pd.concat([df_st1.loc[:,df_st1.columns[df_st1.isna().sum() == 0]].drop('TARGET',axis=1), df_st1[['CODE_GENDER']]],axis=1)

In [ ]:
im = datawig.SimpleImputer(df_gender,
                           'CODE_GENDER')

In [ ]:
im.fit(df_gender,num_epochs=5)

In [ ]:
df_gender = im.predict(df_gender)

In [ ]:
df_gender[df_gender['CODE_GENDER']!=df_gender['CODE_GENDER_imputed']]['CODE_GENDER'].value_counts()


In [ ]:
df_st1['CODE_GENDER'] = df_gender['CODE_GENDER_imputed']

#### 3.1.5 FLAG_OWN_REALTY

- 자가 부동산 보유 여부는 부도확률에 영향을 줄까?

In [ ]:
print(df_st1[df_st1['FLAG_OWN_REALTY']=='Y']['REGION_POPULATION_RELATIVE'].mean())
print(df_st1[df_st1['FLAG_OWN_REALTY']=='N']['REGION_POPULATION_RELATIVE'].mean())

In [ ]:
df_st1 = df_st1.drop('FLAG_OWN_REALTY',axis=1)

- 결측이많고 거의 의미가 없어보인다.

#### 3.1.6 DAYS_EMPLOYED

- DAYS_EMPLOYED 변수는 음수로 표현되어 있으나 365243수치로 이상치 다수 존재

In [ ]:
df_st1['DAYS_EMPLOYED'].value_counts()

- DAYS_EMPLOYED 변수가 365243으로 이상치인 row데이터 중 ORGANIZATION_TYPE 변수는 전부 XNA

In [ ]:
df[df['DAYS_EMPLOYED']==365243]['ORGANIZATION_TYPE'].value_counts()

- 반대의 경우 전부 365243 or NaN값이다.

In [ ]:
df[df['ORGANIZATION_TYPE']== 'XNA']['DAYS_EMPLOYED']

- 마찬가지로 NAME_INCOME_TYPE 변수 또한 전부 Pensioner    

In [ ]:
df[df['DAYS_EMPLOYED']==365243]['NAME_INCOME_TYPE'].value_counts()

In [ ]:
df[df['NAME_INCOME_TYPE']=='Pensioner']['ORGANIZATION_TYPE'].value_counts()

* ORGANIZATION_TYPE 변수가 Pensioner(연금수령자)인 경우 근무일수, 소득유형 타입에서 결측값, 이상값을 보인다.

In [ ]:
df[df['NAME_INCOME_TYPE']=='Pensioner']

- NAME_INCOME_TYPE 변수가 Pensioner 인 경우 DAYS_EMPLOYED 변수는 결측 아니면 365243.0 이다 그러므로 우선 결측치들을 365243으로 채워줌

In [ ]:
df_st1.loc[df_st1['NAME_INCOME_TYPE']=='Pensioner','DAYS_EMPLOYED']=365243.0

In [ ]:
df_st1.loc[:,['DAYS_EMPLOYED','TARGET']].corr()

In [ ]:
df_st1.corr()

- DAY_EMPLOYED 와 상관관계가 존재 하는 것으로 보이는 DAYS_BIRTH, DAYS_ID_PUBLISH를 사용해 예측해보자

In [ ]:
df_st1.loc[:,['NAME_INCOME_TYPE','DAYS_BIRTH']].groupby('NAME_INCOME_TYPE').describe()

- Pensioner(연금수령자)와 Maternity leave(육아휴직)의 수치가 비슷해보인다. 하지만 Maternity leave의 표본이 너무 적다.

In [ ]:
df_st1[((df_st1['NAME_INCOME_TYPE'] =='Pensioner')|(df_st1['NAME_INCOME_TYPE'] =='Working')) & (df_st1['TARGET']==1)].groupby('NAME_INCOME_TYPE').describe()

- 365243도 결측값으로 보고 예측해보자

In [ ]:
df_st1.loc[df_st1['DAYS_EMPLOYED']==365243,'DAYS_EMPLOYED'] =np.NaN


- 너무 많은 결측값이 있으나 근무일수는 중요한 변수로 보인다. 정확도에 집중해 예측 후 사용해 보자

##### 3.1.6.1 DAYS_EMPLOYED datawig대체

In [ ]:
df_st1.iloc[:,[-1]]

In [ ]:
def dl_impute(i):
    dx = pd.concat([df_st1.loc[:,df_st1.columns[df_st1.isna().sum() == 0]].drop('TARGET',axis=1), df_st1[[i]]],axis=1)
    dx = dx.drop(['WEEKDAY_APPR_PROCESS_START','HOUR_APPR_PROCESS_START'],axis=1)
    imp = datawig.SimpleImputer(dx,i)
    imp.fit(dx,num_epochs =5)
    data = imp.predict(dx)
    imp_data = data
    return imp_data

In [ ]:
imp_data = dl_impute('DAYS_EMPLOYED')

In [ ]:
df_st1_dl = df_st1.copy()

In [ ]:
df_st1_dl.loc[df_st1_dl['DAYS_EMPLOYED'].isna()==True,'DAYS_EMPLOYED'] = imp_data.loc[imp_data['DAYS_EMPLOYED'].isna()==True,'DAYS_EMPLOYED_imputed']

In [ ]:
df_st1_dl['DAYS_EMPLOYED'] = df_st1_dl['DAYS_EMPLOYED'].abs()


In [ ]:
plt.boxplot(df_st1_dl['DAYS_EMPLOYED'])
plt.show()

In [ ]:
df_st1_dl['DAYS_EMPLOYED'].hist()

In [ ]:
np.log(df_st1_dl['DAYS_EMPLOYED']).hist()

In [ ]:
df_st1_log['DAYS_EMPLOYED'] = np.log(df_st1['DAYS_EMPLOYED'])

- datawig 대체결과 평균에 영향을 많이 받는 것 처럼 보인다.

##### 3.1.6.2 DAYS_EMPLOYED mice대체

In [ ]:
df_st1_reg = df_st1.copy()

In [ ]:
def mi_impute(i):
    dx = pd.concat([df_st1.loc[:,df_st1.columns[df_st1.isna().sum() == 0]].drop('TARGET',axis=1), df_st1[[i]]],axis=1)
    dx = dx.drop(['WEEKDAY_APPR_PROCESS_START','HOUR_APPR_PROCESS_START'],axis=1)
    dx = pd.get_dummies(dx)
    imp = mice(dx)
    data = pd.DataFrame(imp)
    imp_data = data
    return imp_data

In [ ]:
imp_data_mice = mi_impute('DAYS_EMPLOYED')

In [ ]:
imp_emp_mice = imp_data_mice

In [ ]:
df_st1_reg['DAYS_EMPLOYED'] = imp_data_mice[2].abs()


In [ ]:
df_st1_reg['DAYS_EMPLOYED'].hist()

In [ ]:
np.log(df_st1_reg['DAYS_EMPLOYED'].abs()).hist()

In [ ]:
# df_st1['DAYS_EMPLOYED'] = df_st1_reg['DAYS_EMPLOYED']

- 두방법 모두 거의 평균대체에 가까운 결과가 나온다 결측값이 너무많기 때문에 변수 삭제도 고려해 보자
- 사용한다면 mice기법으로 대체한 값을 사용하자.

#### 3.1.7 AMT_CREDIT

In [ ]:
df_st1[['AMT_CREDIT']].info()

In [ ]:
plt.boxplot(df_st1.loc[df_st1['AMT_CREDIT'].isna()==False,'AMT_CREDIT'])
plt.show()

- AMT_CREDIT 변수는 CB_SCORE_2, AMT_INCOME_TOTAL 과 전혀 무관하지 않은듯 하다.

In [ ]:
df_st1.groupby('NAME_INCOME_TYPE').describe()['AMT_CREDIT']

- 높은 정확도를 위해 datawig의 딥러닝 기법을 이용한 결측값 대체 실시

##### 3.1.7.1 AMT_CREDIT datawig 기법 대체

In [ ]:
imp_cred_dl = dl_impute('AMT_CREDIT')

In [ ]:
df_st1_dl.loc[df_st1_dl['AMT_CREDIT'].isna()==True,'AMT_CREDIT'] = imp_cred_dl.loc[imp_cred_dl['AMT_CREDIT'].isna()==True,'AMT_CREDIT_imputed']

In [ ]:
df_st1_dl['AMT_CREDIT'].hist()

In [ ]:
np.log(df_st1_dl['AMT_CREDIT']).hist()

##### 3.1.7.2 AMT_CREDIT mice 기법 대체

In [ ]:
imp_credit_mice = mi_impute('AMT_CREDIT')
imp_credit_mice

In [ ]:
df_st1_reg['AMT_CREDIT'] = imp_credit_mice[2]

In [ ]:
df_st1_reg['AMT_CREDIT'].hist()

In [ ]:
df_st1['AMT_CREDIT'].hist()

- datawig 변환보단 mice변환이 더 괜찮아 보인다.

In [ ]:
df_st1_log['AMT_CREDIT_reg'] = np.log(df_st1_reg['AMT_CREDIT'])
df_st1_log['AMT_CREDIT_dl'] = np.log(df_st1_dl['AMT_CREDIT'])


#### 3.1.8 REGION_POPULATION_RELATIVE

- 거주지역의 표준화된 인구

In [ ]:
df_st1.corr()

- 결측값도 많고 관계도 찾기 어려워보여 예측이 힘들것같다 삭제해주자.

In [ ]:
df_st1 = df_st1.drop('REGION_POPULATION_RELATIVE',axis=1)

#### 3.1.9 DAYS_BIRTH

In [ ]:
df_st1.corr()

- 상관관계가 보이는 변수들도 있고 상대적으로 결측이 많지 않으니 예측해보자

In [ ]:
imp_birth_dl = dl_impute('DAYS_BIRTH')

In [ ]:
imp_birth_mice = mi_impute('DAYS_BIRTH')

In [ ]:
imp_birth_mice

In [ ]:
df_st1_dl.loc[df_st1_dl['DAYS_BIRTH'].isna()==True,'DAYS_BIRTH'] = imp_birth_dl.loc[imp_birth_dl['DAYS_BIRTH'].isna()==True,'DAYS_BIRTH_imputed']

In [ ]:
df_st1_dl['DAYS_BIRTH'].hist()

In [ ]:
imp_birth_mice[2].hist()

In [ ]:
df_st1_reg['DAYS_BIRTH'] = imp_birth_mice[2]

- mice방법 대체가 더 적절해 보인다.

#### 3.1.8 DAYS_REGISTRATION

In [ ]:
df_st1.corr()

- CB_SCORE2, DAYS_BIRTH 와 상관관계가 있어보인다.
- 하지만 결측값이 많으므로 예측후 분산을 줄이기 위해 그룹화를 해서 사용하자

In [ ]:
imp_regis_dl = dl_impute('DAYS_REGISTRATION')
imp_regis_mice = mi_impute('DAYS_REGISTRATION')

In [ ]:
df_st1_dl.loc[df_st1_dl['DAYS_REGISTRATION'].isna(),'DAYS_REGISTRATION'] = imp_regis_dl.loc[imp_regis_dl['DAYS_REGISTRATION'].isna(),'DAYS_REGISTRATION_imputed']

In [ ]:
df_st1_reg['DAYS_REGISTRATION'] = imp_regis_mice[2]

In [ ]:
df_st1_dl['DAYS_REGISTRATION'].abs().hist()


In [ ]:
df_st1_reg['DAYS_REGISTRATION'].abs().hist()

- datawig, mice 기법모두 지나치게 평균에 영향을 받는것처럼 보인다.

#### 3.1.9 DAYS_ID_PUBLISH

In [ ]:
df_st1.info()

In [ ]:
df_st1.corr()

- 직관적으로 사용이 어려워보인다. 결측도 존재 그러므로 제거해준다.

In [ ]:
df_st1 = df_st1.drop('DAYS_ID_PUBLISH',axis=1)

In [ ]:
df_st1.info()

#### 3.1.10 CB_SCORE_N

In [ ]:
df_st1.corr()

- CB_SCORE 끼리 상관관계가 없다.
- CB_SCORE_2 를 제외하고 다른 변수들과의 상관관계도 보이지 않는다.

In [ ]:
df_st1

In [ ]:
df_reg1 = pd.concat([df_st1.loc[:,df_st1.columns[df_st1.isna().sum() == 0]].drop('TARGET',axis=1),df_st1.loc[:,['CB_SCORE_1', 'CB_SCORE_2','CB_SCORE_3']]],axis=1)

In [ ]:
df_reg1 = df_reg1.drop(['WEEKDAY_APPR_PROCESS_START','HOUR_APPR_PROCESS_START'],axis=1)

In [ ]:
df_reg1_rst= pd.get_dummies(df_reg1)

In [ ]:
imputer = mice(df_reg1_rst)

In [ ]:
reg_1 = pd.DataFrame(imputer)


In [ ]:
df_st1_reg[['CB_SCORE_1','CB_SCORE_2','CB_SCORE_3']] = reg_1.iloc[:,[2,3,4]]

- 각 스코어를 더한 변수를 하나 만들어주자

In [ ]:
df_st1_reg['TOT_CS_SCORE'] = df_st1_reg['CB_SCORE_1']+df_st1_reg['CB_SCORE_2']+df_st1_reg['CB_SCORE_3']
df_st1['TOT_CS_SCORE'] = df_st1_reg['CB_SCORE_1']+df_st1_reg['CB_SCORE_2']+df_st1_reg['CB_SCORE_3']


#### 3.1.10 NAME_FAMILY_STATUS

In [ ]:
df_st1['NAME_FAMILY_STATUS'].value_counts()

In [ ]:
df_st1.loc[df_st1['NAME_FAMILY_STATUS']=='Unknown','NAME_FAMILY_STATUS'] = 'Married'


- 1개의 unknown은 최빈값으로 대체

#### 3.1.11 category가 너무 많은 변수들

- NAME_INCOME_TYPE
- NAME_EDUCATION_TYPE
- NAME_FAMILY_STATUS
- NAME_HOUSING_TYPE
- WEEKDAY_APPR_PROCESS_START
- HOUR_APPR_PROCESS_START

In [ ]:
plot_m = ['NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','WEEKDAY_APPR_PROCESS_START','HOUR_APPR_PROCESS_START']

In [ ]:
fig, ax = plt.subplots(2,3,figsize =(15,10))
a =0
for i in plot_m:
    a_1 = a%3
    a_2 = a//3
    
    ax[a_2,a_1].plot((df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values(),color = 'blue')
    ax[a_2,a_1].set_ylim(0,0.2)
    ax2 = ax[a_2,a_1].twinx()
    sns.countplot(x = i ,palette='Set2' ,data =df_st1 ,order = (df_st1.groupby(i)['TARGET'].sum()/df_st1.groupby(i)['TARGET'].count()).sort_values().index
                  ,ax = ax2)
    ax[a_2,a_1].set_zorder(1)
    ax2.set_zorder(-1)
    ax[a_2,a_1].patch.set_visible(False)
    ax[a_2,a_1].tick_params(axis ='x',
                    rotation = 45,
                   )
    plt.title(i)
    plt.subplots_adjust(hspace = 0.5,
                        wspace = 0.5)
    a +=1

plt.show()

- NAME_EDUCATION_TYPE 변수는 학위 수준을 순위로 라벨링 처리
- WEEKDAY_APPR_PROCESS_START 변수는 의미가 없어 보인다.

In [ ]:
df_st1['NAME_EDUCATION_TYPE'].value_counts()
df_st1 = df_st1.drop('WEEKDAY_APPR_PROCESS_START',axis=1)

- 변수 라벨링

In [ ]:
edu_typeLabel = {'Lower secondary' : 1,
                 'Secondary / secondary special'    : 2,
                 'Incomplete higher'    : 3,
                 'Higher education'   : 4,
                 'Academic degree'        : 5
                }


In [ ]:
df_st1.loc[df_st1['CODE_GENDER']=='F','CODE_GENDER'] = 1
df_st1.loc[df_st1['CODE_GENDER']=='M','CODE_GENDER'] = 0


In [ ]:
df_st1['NAME_EDUCATION_TYPE'] = df_st1['NAME_EDUCATION_TYPE'].apply(lambda x : edu_typeLabel[x])

In [ ]:
df_st1['FLAG_WORK_INFO'] = df_st1['FLAG_WORK_INFO'].astype(int)
df_st1['FLAG_IND_INFO'] = df_st1['FLAG_IND_INFO'].astype(int)
df_st1['FLAG_DOCUMENT_3'] = df_st1['FLAG_DOCUMENT_3'].astype(int)
df_st1['LIVE_CITY_NOT_WORK_CITY'] = df_st1['LIVE_CITY_NOT_WORK_CITY'].astype(int)
df_st1['REG_CITY_NOT_WORK_CITY'] = df_st1['REG_CITY_NOT_WORK_CITY'].astype(int)
df_st1['REG_CITY_NOT_LIVE_CITY'] = df_st1['REG_CITY_NOT_LIVE_CITY'].astype(int)
df_st1['LIVE_CITY_NOT_WORK_CITY'] = df_st1['LIVE_CITY_NOT_WORK_CITY'].astype(int)
df_st1['HOUR_APPR_PROCESS_START'] = df_st1['HOUR_APPR_PROCESS_START'].astype(int)
df_st1['REGION_RATING_CLIENT'] = df_st1['REGION_RATING_CLIENT'].astype(int)
df_st1['HOUR_APPR_PROCESS_START'] = df_st1['HOUR_APPR_PROCESS_START'].astype(int)



In [ ]:
df_st2 = df_st1.copy()

In [ ]:
df_st2.info()

In [ ]:
df_st2['AMT_CREDIT'] = df_st1_reg['AMT_CREDIT']
df_st2['CB_SCORE_1'] = df_st1_reg['CB_SCORE_1']
df_st2['CB_SCORE_2'] = df_st1_reg['CB_SCORE_2']
df_st2['CB_SCORE_3'] = df_st1_reg['CB_SCORE_3']
df_st2['DAYS_BIRTH'] = df_st1_reg['DAYS_BIRTH'].abs()/365
df_st2['DAYS_EMPLOYED'] = df_st1_reg['DAYS_EMPLOYED'].abs()
df_st2['DAYS_REGISTRATION'] = df_st1_reg['DAYS_REGISTRATION'].abs()
df_st2['DAYS_BIRTH_EMP'] = df_st1_reg['DAYS_EMPLOYED'].abs()/df_st1_reg['DAYS_BIRTH'].abs()


- DAYS_BIRTH 를 365로 나누어 나이형태로 변환
- DAYS_BIRTH_EMP : 살아온 기간동안 일한 기간을 비율로 나타내주는 변수

In [ ]:
df_st2 = pd.get_dummies(df_st2)


In [ ]:
MM_scaler = MinMaxScaler()

## modeling

In [ ]:

X = df_st2.drop('TARGET',axis=1)
y = df_st2[['TARGET']].astype(str)


In [ ]:
MM_scaler.fit(X)

In [ ]:
X = MM_scaler.transform(X)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,
                                                    random_state=42,
                                                    stratify =y)

In [ ]:
%%time
model_rf = RandomForestClassifier(random_state=42,n_estimators =10000, max_depth=9)
model_lgb = lgb.LGBMClassifier(random_state=42, learning_rate=0.02, n_estimators=10000,is_unblance = True,boost_from_average = False)
model_xgb = xgb.XGBClassifier(random_state=42, learning_rate = 0.02, n_estimators= 10000)

In [ ]:
%%time
model_rf.fit(X_over,y_over)
model_lgb.fit(X_over,y_over)
model_xgb.fit(X_over,y_over)

In [ ]:
# model_rf.fit(X_train,y_train)
# model_lgb.fit(X_train,y_train)
# model_xgb.fit(X_train,y_train)

In [ ]:
pred_y = model_rf.predict(X_test)
pred_y4 = model_lgb.predict(X_test)
pred_y5 = model_xgb.predict(X_test)


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
print(confusion_matrix(y_test, pred_y), 'auc :' ,roc_auc_score(y_test, pred_y))
print('--------------------------------')

print(confusion_matrix(y_test, pred_y4), roc_auc_score(y_test, pred_y4))
print('--------------------------------')

print(confusion_matrix(y_test, pred_y5), roc_auc_score(y_test, pred_y5))


- 예측이 너무나도 안된다. 오버샘플링을 시도해보자

In [ ]:
sm = SMOTE(random_state = 42)

In [ ]:
X_over, y_over = sm.fit_resample(X_train,y_train)

In [ ]:
%%time
model_rf.fit(X_over,y_over)
model_lgb.fit(X_over,y_over)
model_xgb.fit(X_over,y_over)

In [ ]:
pred_y = model_rf.predict(X_test)
pred_y4 = model_lgb.predict(X_test)
pred_y5 = model_xgb.predict(X_test)


In [ ]:
print(confusion_matrix(y_test, pred_y), 'auc :' ,roc_auc_score(y_test, pred_y))
print('--------------------------------')

print(confusion_matrix(y_test, pred_y4), roc_auc_score(y_test, pred_y4))
print('--------------------------------')

print(confusion_matrix(y_test, pred_y5), roc_auc_score(y_test, pred_y5))

- 조금은 상승된 모습이다. 변수를 처리하거나 만드는 과정에서 결측이 많았던 변수를 빼고 진행해보자

In [ ]:
df_st3 = df_st1.copy()

In [ ]:
df_st3 = df_st3.drop(['AMT_CREDIT','DAYS_EMPLOYED','DAYS_REGISTRATION'],axis=1)

In [ ]:
df_st3['CB_SCORE_1'] = df_st1_reg['CB_SCORE_1']
df_st3['CB_SCORE_2'] = df_st1_reg['CB_SCORE_2']
df_st3['CB_SCORE_3'] = df_st1_reg['CB_SCORE_3']
df_st3['DAYS_BIRTH'] = df_st1_reg['DAYS_BIRTH'].abs()/365


In [ ]:
df_st3 = pd.get_dummies(df_st3)


In [ ]:
MM_scaler = MinMaxScaler()

In [ ]:

X = df_st3.drop('TARGET',axis=1)
y = df_st3[['TARGET']].astype(str)

In [ ]:
MM_scaler.fit(X)

In [ ]:
X = MM_scaler.transform(X)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,
                                                    random_state=42,
                                                    stratify =y)

In [ ]:
sm = SMOTE(random_state = 42)

In [ ]:
X_over, y_over = sm.fit_resample(X_train,y_train)

In [ ]:
%%time
model_rf = RandomForestClassifier(random_state=42,n_estimators =10000, max_depth=9)
model_lgb = lgb.LGBMClassifier(random_state=42, learning_rate=0.02, n_estimators=10000,is_unblance = True,boost_from_average = False)
model_xgb = xgb.XGBClassifier(random_state=42, learning_rate = 0.02, n_estimators= 10000)

In [ ]:
%%time
model_rf.fit(X_over,y_over)
model_lgb.fit(X_over,y_over)
model_xgb.fit(X_over,y_over)

In [ ]:
pred_y = model_rf.predict(X_test)
pred_y4 = model_lgb.predict(X_test)
pred_y5 = model_xgb.predict(X_test)


In [ ]:
print(confusion_matrix(y_test, pred_y), 'auc :' ,roc_auc_score(y_test, pred_y))
print('--------------------------------')

print(confusion_matrix(y_test, pred_y4), roc_auc_score(y_test, pred_y4))
print('--------------------------------')

print(confusion_matrix(y_test, pred_y5), roc_auc_score(y_test, pred_y5))

- 전체적으로 랜덤포레스트가 성능이 제일 좋다.
- 파라미터 튜닝을 해보자
- 변수는 결측을 대체하고 파생변수를 만들었던 df_st2를 사용하자

In [ ]:
MM_scaler = MinMaxScaler()

In [ ]:
X = df_st2.drop('TARGET',axis=1)
y = df_st2[['TARGET']].astype(str)

In [ ]:
MM_scaler.fit(X)

In [ ]:
X = MM_scaler.transform(X)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,
                                                    random_state=42,
                                                    stratify =y)

In [ ]:
sm = SMOTE(random_state = 42)

In [ ]:
X_over, y_over = sm.fit_resample(X_train,y_train)

In [ ]:
from sklearn.model_selection import GridSearchCV

params = { 'n_estimators' : [100, 1000, 10000],
           'max_depth' : [6, 8, 10, 12],
           'min_samples_leaf' : [8, 12, 18],
           'min_samples_split' : [8, 16, 20]
            }

# RandomForestClassifier 객체 생성 후 GridSearchCV 수행
modrl_rf_gscv = RandomForestClassifier(random_state = 0, n_jobs = -1)
grid_cv = GridSearchCV(modrl_rf_gscv, param_grid = params, cv = 4, n_jobs = -1)
grid_cv.fit(X_over, y_over)

- 전처리함수

In [ ]:
def makeX(data):
    df_st1 = data.copy()
    n_col=[]
    for i in df_st1.columns:
        n_col.append(i.upper())
    df_st1.columns = n_col
    df_st1['FLAG_MOBIL'] = df_st1['FLAG_MOBIL'].astype('str')
    df_st1['FLAG_EMP_PHONE'] = df_st1['FLAG_EMP_PHONE'].astype('str')
    df_st1['FLAG_WORK_PHONE'] = df_st1['FLAG_WORK_PHONE'].astype('str')
    df_st1['FLAG_PHONE'] = df_st1['FLAG_PHONE'].astype('str')
    df_st1['FLAG_EMAIL'] = df_st1['FLAG_EMAIL'].astype('str')
    df_st1['REGION_RATING_CLIENT'] = df_st1['REGION_RATING_CLIENT'].astype('str')
    df_st1['HOUR_APPR_PROCESS_START'] = df_st1['HOUR_APPR_PROCESS_START'].astype('str')
    df_st1['REG_CITY_NOT_LIVE_CITY'] = df_st1['REG_CITY_NOT_LIVE_CITY'].astype('str')
    df_st1['REG_CITY_NOT_WORK_CITY'] = df_st1['REG_CITY_NOT_WORK_CITY'].astype('str')
    df_st1['LIVE_CITY_NOT_WORK_CITY'] = df_st1['LIVE_CITY_NOT_WORK_CITY'].astype('str')
    df_st1['FLAG_DOCUMENT_2'] = df_st1['FLAG_DOCUMENT_2'].astype('str')
    df_st1['FLAG_DOCUMENT_3'] = df_st1['FLAG_DOCUMENT_3'].astype('str')
    df_st1['FLAG_DOCUMENT_4'] = df_st1['FLAG_DOCUMENT_4'].astype('str')
    df_st1['CNT_CHILDREN']               = df_st1['CNT_CHILDREN'].astype('int')
    df_st1['AMT_INCOME_TOTAL']           = df_st1['AMT_INCOME_TOTAL'].astype('float64')
    df_st1['AMT_CREDIT']                 = df_st1['AMT_CREDIT'].astype('float64')
    df_st1['REGION_POPULATION_RELATIVE'] = df_st1['REGION_POPULATION_RELATIVE'].astype('float64')
    df_st1['DAYS_BIRTH']                 = df_st1['DAYS_BIRTH'].astype('float64')
    df_st1['DAYS_EMPLOYED']              = df_st1['DAYS_EMPLOYED'].astype('float64')
    df_st1['DAYS_REGISTRATION']          = df_st1['DAYS_REGISTRATION'].astype('float64')
    df_st1['DAYS_ID_PUBLISH']            = df_st1['DAYS_ID_PUBLISH'].astype('float64')
    df_st1['DAYS_EMPLOYED']              = df_st1['DAYS_EMPLOYED'].astype('float64')
    df_st1['CB_SCORE_1']                 = df_st1['CB_SCORE_1'].astype('float64')
    df_st1['CB_SCORE_2']                 = df_st1['CB_SCORE_2'].astype('float64')
    df_st1['CB_SCORE_3']                 = df_st1['CB_SCORE_3'].astype('float64')
    df_st1['TARGET'] = df_st1['TARGET'].astype('int')
    
    
    
    
    
    
    df_st1 = df_st1.drop(['SK_ID_CURR','FLAG_MOBIL','FLAG_DOCUMENT_2','FLAG_DOCUMENT_4'],axis=1)
    
    
    df_st1.loc[df_st1['AMT_INCOME_TOTAL']>=1000000,'AMT_INCOME_TOTAL'] = 1000000
    
    
    df_st1['ORGANIZATION_TYPE2'] = df_st1['ORGANIZATION_TYPE'].str.split().str[0]
    df_st1['ORGANIZATION_TYPE3'] = ''
    a = ['Religion','Police','Realtor','Culture','Insurance','Mobile','Services','Military','Postal','Cleaning','Bank','University']
    b = ['XNA']
    c = ['Medicine','School','Hotel','Kindergarten','Advertising','Telecom']
    d = ['Business']
    e = ['Restaurant','Government','Transport:','Other','Industry:','Trade:']
    f = ['Self-employed']
    g = ['Construction','Security','Housing','Electricity','Agriculture','Emergency','Legal']
    
    df_st1.loc[df_st1.ORGANIZATION_TYPE2.isin(a),'ORGANIZATION_TYPE3'] = 'group1'
    df_st1.loc[df_st1.ORGANIZATION_TYPE2.isin(b),'ORGANIZATION_TYPE3'] = 'group2'
    df_st1.loc[df_st1.ORGANIZATION_TYPE2.isin(c),'ORGANIZATION_TYPE3'] = 'group3'
    df_st1.loc[df_st1.ORGANIZATION_TYPE2.isin(d),'ORGANIZATION_TYPE3'] = 'group4'
    df_st1.loc[df_st1.ORGANIZATION_TYPE2.isin(e),'ORGANIZATION_TYPE3'] = 'group5'
    df_st1.loc[df_st1.ORGANIZATION_TYPE2.isin(f),'ORGANIZATION_TYPE3'] = 'group6'
    df_st1.loc[df_st1.ORGANIZATION_TYPE2.isin(g),'ORGANIZATION_TYPE3'] = 'group7'
    df_st1.loc[df_st1.ORGANIZATION_TYPE3== '','ORGANIZATION_TYPE3'] = 'group4'

    df_st1 = df_st1.drop(['ORGANIZATION_TYPE','ORGANIZATION_TYPE2'],axis=1)
    
    
    
    df_st1['FLAG_WORK_INFO'] = df_st1['FLAG_EMP_PHONE'].astype(int) + df_st1['FLAG_WORK_PHONE'].astype(int)
    df_st1['FLAG_WORK_INFO'] = df_st1['FLAG_WORK_INFO'].astype(str)
    df_st1['FLAG_IND_INFO'] = df_st1['FLAG_PHONE'].astype(int) + df_st1['FLAG_EMAIL'].astype(int)
    df_st1['FLAG_IND_INFO'] = df_st1['FLAG_IND_INFO'].astype(str)
    
    df_st1 = df_st1.drop(['FLAG_EMP_PHONE','FLAG_WORK_PHONE','FLAG_PHONE','FLAG_EMAIL'],axis=1)
    
    
    df_st1.loc[df_st1['CODE_GENDER']=='XNA','CODE_GENDER'] = np.NaN
    if df_st1['CODE_GENDER'].isna().sum() >0 :
        df_gender =pd.concat([df_st1.loc[:,df_st1.columns[df_st1.isna().sum() == 0]].drop('TARGET',axis=1), df_st1[['CODE_GENDER']]],axis=1)
        im = datawig.SimpleImputer(df_gender,
                               'CODE_GENDER')
        im.fit(df_gender,num_epochs=5)
        df_gender = im.predict(df_gender)
        df_st1['CODE_GENDER'] = df_gender['CODE_GENDER_imputed']
        
        
        
        
    df_st1 = df_st1.drop('FLAG_OWN_REALTY',axis=1)
    df_st1 = df_st1.drop('REGION_POPULATION_RELATIVE',axis=1)
    
    
    df_st1['DAYS_BIRTH'] = df_st1['DAYS_BIRTH'].abs()
    df_st1 = df_st1.drop('DAYS_ID_PUBLISH',axis=1)
    df_st1.loc[df_st1['NAME_FAMILY_STATUS']=='Unknown','NAME_FAMILY_STATUS'] = 'Married'
    
    
    df_st1 = df_st1.drop('WEEKDAY_APPR_PROCESS_START',axis=1)
    df_st1.loc[df_st1['CODE_GENDER']=='F','CODE_GENDER'] = 1
    df_st1.loc[df_st1['CODE_GENDER']=='M','CODE_GENDER'] = 0
    edu_typeLabel = {'Lower secondary' : 1,
                 'Secondary / secondary special'    : 2,
                 'Incomplete higher'    : 3,
                 'Higher education'   : 4,
                 'Academic degree'        : 5
                }
    df_st1.loc[df_st1['NAME_FAMILY_STATUS']=='Unknown','NAME_FAMILY_STATUS'] = 'Married'
    df_st1['NAME_EDUCATION_TYPE'] = df_st1['NAME_EDUCATION_TYPE'].apply(lambda x : edu_typeLabel[x])
    df_st1['FLAG_WORK_INFO'] = df_st1['FLAG_WORK_INFO'].astype(int)
    df_st1['FLAG_IND_INFO'] = df_st1['FLAG_IND_INFO'].astype(int)
    df_st1['FLAG_DOCUMENT_3'] = df_st1['FLAG_DOCUMENT_3'].astype(int)
    df_st1['LIVE_CITY_NOT_WORK_CITY'] = df_st1['LIVE_CITY_NOT_WORK_CITY'].astype(int)
    df_st1['REG_CITY_NOT_WORK_CITY'] = df_st1['REG_CITY_NOT_WORK_CITY'].astype(int)
    df_st1['REG_CITY_NOT_LIVE_CITY'] = df_st1['REG_CITY_NOT_LIVE_CITY'].astype(int)
    df_st1['LIVE_CITY_NOT_WORK_CITY'] = df_st1['LIVE_CITY_NOT_WORK_CITY'].astype(int)
    df_st1['HOUR_APPR_PROCESS_START'] = df_st1['HOUR_APPR_PROCESS_START'].astype(int)
    df_st1['REGION_RATING_CLIENT'] = df_st1['REGION_RATING_CLIENT'].astype(int)
    df_st1['HOUR_APPR_PROCESS_START'] = df_st1['HOUR_APPR_PROCESS_START'].astype(int)
    df_st1['DAYS_BIRTH_EMP'] = df_st1['DAYS_EMPLOYED'].abs()/df_st1['DAYS_BIRTH'].abs()
    df_st1['DAYS_BIRTH'] = df_st1['DAYS_BIRTH'].abs()/365
    df_st1['DAYS_EMPLOYED'] = df_st1['DAYS_EMPLOYED'].abs()
    df_st1['TOT_CS_SCORE'] = df_st1['CB_SCORE_1'] +df_st1['CB_SCORE_2'] + df_st1['CB_SCORE_3']
    df_st1['DAYS_REGISTRATION'] = df_st1['DAYS_REGISTRATION'].abs()
    df_st1 = pd.get_dummies(df_st1)
    
    
    return df_st1

In [ ]:
dfnn = makeX(df)
X = dfnn.drop('TARGET',axis=1)
X = MM_scaler.transform(X)
y = dfnn[['TARGET']]